In [13]:
import tensorflow.keras as keras
import os
import random
import re
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import datetime

import tensorflow
%load_ext tensorboard
import datetime, os

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
DATA_PATH = '../Data/'
FRAME_PATH = DATA_PATH+'Images/'
MASK_PATH = DATA_PATH+'Masks/'

DATASET_PATH = DATA_PATH + 'Dataset/'

# Create folders to hold images and masks
folders = ['train_frames/train', 'train_masks/train', 
           'val_frames/val', 'val_masks/val', 
           'test_frames/test', 'test_masks/test']

In [3]:
all_frames = os.listdir(FRAME_PATH)
all_masks = os.listdir(MASK_PATH)
random.shuffle(all_frames)


# Generate train, val, and test sets for frames

train_split = int(0.7*len(all_frames))
val_split = int(0.9 * len(all_frames))

train_frames = all_frames[:train_split]
val_frames = all_frames[train_split:val_split]
test_frames = all_frames[val_split:]

# Generate corresponding mask lists for masks

train_masks = [f for f in all_masks if f in train_frames]
val_masks = [f for f in all_masks if f in val_frames]
test_masks = [f for f in all_masks if f in test_frames]

In [4]:
#Add train, val, test frames and masks to relevant folders


def add_frames(dir_name, image): 
    img = Image.open(FRAME_PATH+image)
    img.save(DATASET_PATH+'{}'.format(dir_name)+'/'+image)
    
def add_masks(dir_name, image):
    img = Image.open(MASK_PATH+image)
    img.save(DATASET_PATH+'{}'.format(dir_name)+'/'+image)

    
frame_folders = [(train_frames, 'train_frames/train'), (val_frames, 'val_frames/val'), 
                 (test_frames, 'test_frames/test')]

mask_folders = [(train_masks, 'train_masks/train'), (val_masks, 'val_masks/val'), 
                (test_masks, 'test_masks/test')]

# Add frames

for folder in frame_folders:
    array = folder[0]
    name = [folder[1]] * len(array)

    list(map(add_frames, name, array))
         
    
# Add masks

for folder in mask_folders:
    array = folder[0]
    name = [folder[1]] * len(array)
    list(map(add_masks, name, array))

In [4]:
train_datagen = ImageDataGenerator(
        rescale=1./255)
        #shear_range=0.2,
        #zoom_range=0.2,
        #horizontal_flip=True)
        
val_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
batch_size = 4
train_image_generator = train_datagen.flow_from_directory(DATASET_PATH + 'train_frames', batch_size = batch_size)
train_mask_generator = train_datagen.flow_from_directory(DATASET_PATH + 'train_masks',batch_size = batch_size)
val_image_generator = val_datagen.flow_from_directory(DATASET_PATH + 'val_frames',batch_size = batch_size)
val_mask_generator = val_datagen.flow_from_directory(DATASET_PATH + 'val_masks',batch_size = batch_size)

train_generator = zip(train_image_generator, train_mask_generator)
val_generator = zip(val_image_generator, val_mask_generator)

Found 1992 images belonging to 1 classes.
Found 1992 images belonging to 1 classes.
Found 800 images belonging to 1 classes.
Found 800 images belonging to 1 classes.


In [6]:
# Build U-Net model
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
 
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
 
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c5)
 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#model.summary()

In [7]:
import cv2

def data_gen(img_folder, mask_folder, batch_size):
    c = 0
    n = os.listdir(img_folder) #List of training images
    random.shuffle(n)
  
    while (True):
        img = np.zeros((batch_size, 256, 256, 3)).astype('float')
        mask = np.zeros((batch_size, 256, 256, 1)).astype('float')

        for i in range(c, c+batch_size): #initially from 0 to 16, c = 0. 

            train_img = cv2.imread(img_folder+'/'+n[i])/255.
            train_img =  cv2.resize(train_img, (256, 256))# Read an image from folder and resize
      
            img[i-c] = train_img #add to array - img[0], img[1], and so on.
                                                   

            train_mask = cv2.imread(mask_folder+'/'+n[i], cv2.IMREAD_GRAYSCALE)/255.
            train_mask = cv2.resize(train_mask, (256, 256))
            train_mask = train_mask.reshape(256, 256, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]

            mask[i-c] = train_mask

        c+=batch_size
        if(c+batch_size>=len(os.listdir(img_folder))):
            c=0
            random.shuffle(n)
                  # print "randomizing again"
        yield img, mask




train_frame_path = DATASET_PATH + 'train_frames/train'
train_mask_path = DATASET_PATH + 'train_masks/train'

val_frame_path = DATASET_PATH + 'val_frames/val'
val_mask_path = DATASET_PATH + 'val_masks/val'

# Train the model
train_gen = data_gen(train_frame_path,train_mask_path, batch_size = 4)
val_gen = data_gen(val_frame_path,val_mask_path, batch_size = 4)

In [17]:
logdir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir,write_images=True)

NO_OF_TRAINING_IMAGES = len(os.listdir(DATASET_PATH + 'train_frames/train'))
NO_OF_VAL_IMAGES = len(os.listdir(DATASET_PATH + 'val_frames/val'))

NO_OF_EPOCHS = 30

BATCH_SIZE = batch_size

weights_path = 'weigths'

#opt = Adam(lr=1E-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

#checkpoint = ModelCheckpoint(weights_path, monitor='binary_crossentropy', 
#                             verbose=1, save_best_only=True, mode='max')

#earlystopping = EarlyStopping(monitor = 'binary_crossentropy', verbose = 1,
#                              min_delta = 0.01, patience = 3, mode = 'max')

callbacks_list = [tensorboard_callback]#csv_logger, tbCallBack]

results = model.fit_generator(train_gen, epochs=NO_OF_EPOCHS, 
                          steps_per_epoch = (NO_OF_TRAINING_IMAGES//BATCH_SIZE),
                          validation_data=val_gen, 
                          validation_steps=(NO_OF_VAL_IMAGES//BATCH_SIZE), 
                          callbacks=callbacks_list)
model.save('Model.h5')

Epoch 1/30
498/498 [==============================] - 54s 108ms/step - loss: 0.1483 - acc: 0.9416 - val_loss: 0.0879 - val_acc: 0.9665
Epoch 2/30
498/498 [==============================] - 51s 103ms/step - loss: 0.0967 - acc: 0.9652 - val_loss: 0.0743 - val_acc: 0.9728
Epoch 3/30
498/498 [==============================] - 52s 105ms/step - loss: 0.0769 - acc: 0.9728 - val_loss: 0.0915 - val_acc: 0.9675
Epoch 4/30
498/498 [==============================] - 52s 103ms/step - loss: 0.0773 - acc: 0.9728 - val_loss: 0.0669 - val_acc: 0.9765
Epoch 5/30
498/498 [==============================] - 51s 103ms/step - loss: 0.0704 - acc: 0.9754 - val_loss: 0.0655 - val_acc: 0.9782
Epoch 6/30
498/498 [==============================] - 52s 103ms/step - loss: 0.0699 - acc: 0.9758 - val_loss: 0.0612 - val_acc: 0.9780
Epoch 7/30
498/498 [==============================] - 51s 102ms/step - loss: 0.0657 - acc: 0.9771 - val_loss: 0.0577 - val_acc: 0.9791
Epoch 8/30
498/498 [==============================] - 5